In [ ]:
import networkx as nx

In [ ]:
edgotype_train = nx.read_gpickle("data/y2hEdgotyping/edgotype_train.gpickle")

In [ ]:
next(iter(edgotype_train.edges(data=True)))

In [ ]:
edgotype_train.nodes["ENSG00000155755"]

In [ ]:
edgotype_train.nodes["ENSG00000134049"]["seq"][:77] + "P" + edgotype_train.nodes["ENSG00000134049"]["seq"][78:]

In [ ]:
edgotype_train.nodes["ENSG00000134049"]["seq"]

In [ ]:
n2e = {ensg: {"structure":edgotype_train.nodes[ensg]["alphafoldStructures"],
              "mutations":[]} for ensg in edgotype_train.nodes()}

In [ ]:
for u,v,e in edgotype_train.edges(data=True):
    k = e["db_ensembl_gene_id_mt"]
    n2e[k]["mutations"].append(e["Substitution"])

In [ ]:
n2e = {k:{"mutations":v["mutations"],
         "structure":v["structure"][0]} for k,v in n2e.items() if len(v["mutations"]) and len(v["structure"])}

In [ ]:
from Bio.PDB import PDBParser

In [ ]:
parser = PDBParser()

In [ ]:
import gzip

In [ ]:
for k,v in n2e.items():
    with gzip.open(v["structure"],"rt") as f:
        struc =parser.get_structure("AF-O75800", f)
        n2e[k]["chains"] = [c.id for c in struc.get_chains()][0]

In [ ]:
mutfile = lambda loc,chain,aa_mut:f"""NATRO
start
{loc} {chain} PIKAA {aa_mut}
"""

In [ ]:
import os

In [ ]:
from shutil import move,copy

In [ ]:
import sys

In [ ]:
import subprocess

In [ ]:
from tqdm import tqdm

In [ ]:
for k,v in tqdm(n2e.items(),total=len(n2e)):
    for mutation in set(v["mutations"]):
        pth = f"/home/dzeiberg/flex_ddG_tutorial/ppi_inputs/{k}_{mutation[:-1]}_{mutation[-1]}"
        if not os.path.isdir(pth):
            os.mkdir(pth)
        with open(os.path.join(pth,"chains_to_move.txt"),"w") as f:
            f.write(v["chains"])
#         copy(v["structure"],pth)
        sf = v["structure"]
        of = os.path.join(pth,f"{k}.pdb")
        os.system(f"gunzip -c {sf} > {of}")
        os.chdir(pth)
        os.system(f"/data/utilities/bio/rosetta/rosetta.source.release-333/main/tools/protein_tools/scripts/clean_pdb.py {of} A")
        os.system(f"rm {of}")
        with open(os.path.join(pth,"nataa_mutations.resfile"),"w") as f:
            f.write(mutfile(mutation[1:-1],v["chains"],mutation[-1]))